In [2]:
import sys
import psycopg2
import requests


In [3]:
table = 'record'


In [32]:
conn = psycopg2.connect("dbname='ministries' user='postgres' host='193.85.191.172' password='oracle'")

with conn.cursor() as cur:

    select_rows_stm = '''SELECT t1.record_id, t1.product_name, t1.downloaded
    FROM {table}_product_list t1
    LEFT OUTER JOIN {table}_contracts t2 ON t1.record_id = t2.record_id
    WHERE t2.record_id IS NULL'''.format(table=table)
    cur.execute(select_rows_stm)
    rows = cur.fetchall()

    for idx, row in enumerate(rows, start=1):
        row_id = row[0]
        text = row[1]
        downloaded = row[2]

        print(text)
        if downloaded is not True:
            r = requests.get(url='https://www.hlidacstatu.cz/api/v1/search',
                             params={'query': 'textSmlouvy:"{}"'.format(text)}, headers={'Authorization': 'Token ' + 'dba0c6d0c73d4f45ba8fe929b39fe637'})
            data = r.json()
            print(len(data['items']))
            for item in data['items']:
                contract = {'record_id': row_id,
                            'sign_date': item['datumUzavreni'],
                            'payer': item['Platce']['ico'],
                            'supplier': item['Prijemce'][0]['ico'],
                            'description': item['predmet'],
                            'amount': item['CalculatedPriceWithVATinCZK'],
                            'link': 'https://www.hlidacstatu.cz/Detail/{}'.format(item['identifikator']['idVerze'])}
                insert_stm = '''INSERT INTO {table}_contracts (record_id, sign_date, payer, supplier, description, amount, link) 
                VALUES (%s, %s, %s, %s, %s, %s, %s)'''.format(table=table)
                cur.execute(insert_stm, list(contract.values()))
            update_stm = '''UPDATE {table}_product_list SET downloaded=True WHERE record_id=%s'''.format(table=table)
            cur.execute(update_stm, [row_id])
    conn.commit()
        

Hicom 300
28
Hicom HiPath
1
HP 8460 p S / N
0
Pioneer VSX 9511
0
Hubex L
19
Hubex L , K
19
Intellex DVMS
5
INTIMUS 120 CC
2
INTIMUS 32 CC
2
HSM 102.2
5
Hydrotherm Stiebel MV 180
0
IDS Tipping Point
0
IDSIMA 4
3
IOP 2015
1
PDA - N 8 - 00 HP
0
PH 5 Plus
0
PIXMA MP 640
40
PK 1000
40
Portaro 1
0
HP Elite 3
0
HP LJ CP 6015
0
HP OFFICEJET 100 MOBILE 3302 / 037
0
HP SAS 146 GB
0
HTC 1
13
HTC 4
0
HTC Desire 310
1
HTC HD 2
0
HU 87 - 10004885
0
Huawei E 5180
0
Hyundai VK 222
17
IBM 42 U
2
Panasonic PT - D 5700
0
Philips MCM 1150
0
Photoshop
40
Alcatel Onetouch
3
senior developer
14
Acrobat 11 Pro CZ NEW GOV
0
DVD DX 8000
0
HP 300 G HPCE 410 X + CF 370 AM CZ 10 NF
0
HP 5000 LC
0
hp 5225
2
HP 846
1
Proma BKL 1500
0
PRONEC 2011
0
RAPESCO
10
RAPID 5080
0
Rapiscan 620 XR
2
DVR Intellex DVMS
5
E 4500
4
EB 100
40
EB 600
40
EBA - 2226 S
3
ECG MTM Goddess RSC 085 GW 8 S
0
ECG MTM VENUS RM 140 A + )
0
EHP - FMN 2 CZ 10
0
EMC NetWorker
40
EMC VMAX 10
0
ENVISHOP 2016
0
ALI 1520
2
AD 87
4
Hiross
21
HiSCAN 60

In [27]:
r = requests.get(url='https://www.hlidacstatu.cz/api/v1/search',
                 params={'query': 'textSmlouvy:"iPhone 4 S"'}, headers={'Authorization': 'Token ' + 'dba0c6d0c73d4f45ba8fe929b39fe637'})
data = r.json()

In [30]:
data['items'][0]

{'identifikator': {'idSmlouvy': '3942828', 'idVerze': '4252628'},
 'schvalil': None,
 'hodnotaBezDph': 82319.0,
 'hodnotaVcetneDph': 99606.0,
 'ciziMena': None,
 'platnyZaznam': True,
 'PravniRamec': 1,
 'Prilohy': None,
 'LastUpdate': '2018-05-20T14:17:09.2437192+02:00',
 'CalculatedPriceWithVATinCZK': 99606.0,
 'CalcutatedPriceQuality': 1,
 'Classification': {'LastUpdate': None, 'Version': 0, 'Types': None},
 'Issues': [],
 'Enhancements': [{'Created': '2017-12-29T15:36:43.1431941+01:00',
   'Title': 'Text přílohy extrahován přímo z dokumentu',
   'Description': '',
   'Changed': {'ParameterName': 'item.Prilohy[0].PlainTextContent',
    'PreviousValue': '',
    'NewValue': ''},
   'Public': True,
   'EnhancerType': 'HlidacSmluv.Plugin.Enhancers.TextMiner'}],
 'ConfidenceValue': 0.0,
 'Id': '4252628',
 'odkaz': 'https://smlouvy.gov.cz/smlouva/4252628',
 'casZverejneni': '2017-12-29T15:07:25+01:00',
 'VkladatelDoRejstriku': {'datovaSchranka': '5qt8sy8',
  'nazev': 'Národní knihovna Čes

In [26]:
r.text

'{"total":16,"items":[{"identifikator":{"idSmlouvy":"3942828","idVerze":"4252628"},"schvalil":null,"hodnotaBezDph":82319.0,"hodnotaVcetneDph":99606.0,"ciziMena":null,"platnyZaznam":true,"PravniRamec":1,"Prilohy":null,"LastUpdate":"2018-05-20T14:17:09.2437192+02:00","CalculatedPriceWithVATinCZK":99606.0,"CalcutatedPriceQuality":1,"Classification":{"LastUpdate":null,"Version":0,"Types":null},"Issues":[],"Enhancements":[{"Created":"2017-12-29T15:36:43.1431941+01:00","Title":"Text přílohy extrahován přímo z dokumentu","Description":"","Changed":{"ParameterName":"item.Prilohy[0].PlainTextContent","PreviousValue":"","NewValue":""},"Public":true,"EnhancerType":"HlidacSmluv.Plugin.Enhancers.TextMiner"}],"ConfidenceValue":0.0,"Id":"4252628","odkaz":"https://smlouvy.gov.cz/smlouva/4252628","casZverejneni":"2017-12-29T15:07:25+01:00","VkladatelDoRejstriku":{"datovaSchranka":"5qt8sy8","nazev":"Národní knihovna České republiky","ico":"00023221","adresa":"Klementinum 190 11001, Praha Hlavní město Pr